<a href="https://colab.research.google.com/github/componavt/sns4human/blob/main/src/vk/nlp/topic_modeling2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)

from nltk.corpus import stopwords
stop_words = stopwords.words("russian")
csv_files = [
        'https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/posts/aparfenchikov.csv',
        'https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/posts/minnazrk.csv'
    ]
df = pd.concat([pd.read_csv(url, usecols = ['text']) for url in csv_files], ignore_index=True)
df = df[df['text'].apply(lambda x: isinstance(x, str))]
df = df.reset_index()
df = df.drop(columns=['index'])

#df

In [2]:
!pip install -U pymorphy2==0.9.1
import pymorphy2
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
morph = pymorphy2.MorphAnalyzer(lang='ru')
alphabet = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя-')

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 944.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 11.4 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=68317182d2a0990bff20b9be3356a1a8c32956d30b1548a36e0762386e6d43d6
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [3]:
def process_text(text):
    check_hash = False
    processed_parts = []
    for w in nltk.word_tokenize(text):
      if len(w) == 1:
        continue
      if w == '#':
          check_hash = True
          continue
      if check_hash:
          check_hash = False
          continue
      w_tag = morph.parse(w.strip())[0].tag
      if   'Surn' in w_tag or 'Name' in w_tag or 'Patr' in w_tag:
         continue
      if set(w.lower()).issubset(alphabet):
        if w.isalpha() and w.lower():
          if w.isupper() and len(w) <= 3:
              processed_parts.append(w)
          else:
              res = morph.parse(w.lower())[0].normal_form
              if res not in stop_words:
                  processed_parts.append(res)
    result = ' '.join(processed_parts)
    return str(result)

df['tokens'] = df['text'].apply(lambda x: process_text(x))
# Сохраняем обработанные данные
df.to_excel('df_tokens.xlsx', index=False)
#df

In [4]:
data = pd.read_excel('df_tokens.xlsx')['tokens'].to_string(index=False)


In [5]:
data = [[ doc ] for doc in data.split('\n')]

In [ ]:
data

In [7]:
new_data = [[word for word in sent[0].split()] for sent in data]

In [8]:
from gensim import corpora

# Пример документов
documents = [
    ['apple', 'banana', 'apple'],
    ['banana', 'orange', 'fruit'],
    ['fruit', 'apple', 'fruit']
]

# Создание словаря
dictionary = corpora.Dictionary(documents)

# Просмотр содержимого словаря
print(dictionary.token2id)  # Словарь, где ключи - слова, а значения - уникальные идентификаторы


{'apple': 0, 'banana': 1, 'fruit': 2, 'orange': 3}


In [9]:
for sent in new_data:
    for word in sent:
      if len(word) == 1:
        print(word)

In [10]:
bigram = gensim.models.Phrases(data, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

data_words_trigrams = make_trigrams(data)

dictionary = corpora.Dictionary(data_words_trigrams)
dictionary.filter_extremes(no_below=20, no_above=0.6)

в этом моменте, одинаковые значения выводятся и между ними нет связей, по какой то причине к словам добавляются троеточие и непонятно по какой причине

In [11]:
from gensim.models import LdaModel

dictionary = corpora.Dictionary(new_data)
corpus = [dictionary.doc2bow(text) for text in data]
# Обучение модели LDA
num_topics = 10

lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

# Вывод тем и связанных с ними слов
for idx, topic in lda_model.print_topics(-1):
    print(f"Тема {idx + 1}: {topic}")

/usr/local/lib/python3.10/dist-packages/gensim/models/ldamodel.py:847: RuntimeWarning: invalid value encountered in scalar divide
  perwordbound = self.bound(chunk, subsample_ratio=subsample_ratio) / (subsample_ratio * corpus_words)


Тема 1: 0.000*"режим..." + 0.000*"автономный" + 0.000*"рассмотрение" + 0.000*"ассоциация" + 0.000*"диалог..." + 0.000*"ведение" + 0.000*"средний..." + 0.000*"казначейство" + 0.000*"положител..." + 0.000*"консерв..."
Тема 2: 0.000*"режим..." + 0.000*"автономный" + 0.000*"рассмотрение" + 0.000*"ассоциация" + 0.000*"диалог..." + 0.000*"ведение" + 0.000*"средний..." + 0.000*"казначейство" + 0.000*"положител..." + 0.000*"консерв..."
Тема 3: 0.000*"режим..." + 0.000*"автономный" + 0.000*"рассмотрение" + 0.000*"ассоциация" + 0.000*"диалог..." + 0.000*"ведение" + 0.000*"средний..." + 0.000*"казначейство" + 0.000*"положител..." + 0.000*"консерв..."
Тема 4: 0.000*"режим..." + 0.000*"автономный" + 0.000*"рассмотрение" + 0.000*"ассоциация" + 0.000*"диалог..." + 0.000*"ведение" + 0.000*"средний..." + 0.000*"казначейство" + 0.000*"положител..." + 0.000*"консерв..."
Тема 5: 0.000*"режим..." + 0.000*"автономный" + 0.000*"рассмотрение" + 0.000*"ассоциация" + 0.000*"диалог..." + 0.000*"ведение" + 0.000*

In [104]:
topics = []
for i in range(1, num_topics + 1):
    topics.append(f"Тема {i}")
data = {'Слова': []}

In [14]:
print(lda_model.show_topics(formatted=False))

[(0, [('режим...', 8.461667e-05), ('автономный', 8.461667e-05), ('рассмотрение', 8.461667e-05), ('ассоциация', 8.461667e-05), ('диалог...', 8.461667e-05), ('ведение', 8.461667e-05), ('средний...', 8.461667e-05), ('казначейство', 8.461667e-05), ('положител...', 8.461667e-05), ('консерв...', 8.461667e-05)]), (1, [('режим...', 8.461667e-05), ('автономный', 8.461667e-05), ('рассмотрение', 8.461667e-05), ('ассоциация', 8.461667e-05), ('диалог...', 8.461667e-05), ('ведение', 8.461667e-05), ('средний...', 8.461667e-05), ('казначейство', 8.461667e-05), ('положител...', 8.461667e-05), ('консерв...', 8.461667e-05)]), (2, [('режим...', 8.461667e-05), ('автономный', 8.461667e-05), ('рассмотрение', 8.461667e-05), ('ассоциация', 8.461667e-05), ('диалог...', 8.461667e-05), ('ведение', 8.461667e-05), ('средний...', 8.461667e-05), ('казначейство', 8.461667e-05), ('положител...', 8.461667e-05), ('консерв...', 8.461667e-05)]), (3, [('режим...', 8.461667e-05), ('автономный', 8.461667e-05), ('рассмотрение'

In [15]:
topics = [[(term, round(wt, 3))
               for term, wt in lda_model.show_topic(n)]
                   for n in range(0, lda_model.num_topics)]
topics_df = pd.DataFrame([[term for term, wt in topic]
                              for topic in topics],
                         columns = ['Term'+str(i) for i in range(1, 11)],
                         index=['Topic '+str(t) for t in range(1, lda_model.num_topics+1)]).T
topics_df[:2]

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10
Term1,режим...,режим...,режим...,режим...,режим...,режим...,режим...,режим...,режим...,режим...
Term2,автономный,автономный,автономный,автономный,автономный,автономный,автономный,автономный,автономный,автономный


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = df_state['Частотные слова'].tolist()
print(corpus)
count_vect = CountVectorizer(binary=True)
dataset = count_vect.fit_transform(corpus)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
n_topics = 5
topic_labels = ['Тема {}'.format(i) for i in range(1, n_topics + 1)]
lda = LatentDirichletAllocation(n_components=n_topics, n_jobs=-1, max_iter=500,
                                learning_method='batch', evaluate_every=5,
                                verbose=1, random_state=42)
lda.fit(dataset)

In [ ]:
# Печать тем
for idx, topic in enumerate(lda.components_):
    print(f"Topic {idx}:")
    print([count_vect.get_feature_names_out()[i] for i in topic.argsort()[-5:]])

In [ ]:
topics_count = lda.components_
topics_prob = topics_count / topics_count.sum(axis=1).reshape(-1, 1)
topics = pd.DataFrame(topics_prob.T,
                      index=words,
                      columns=topic_labels)
topics[topics.gt(0).all(1)].shape[0] == topics.shape[0]
n_words = 12
top_words = {}
for topic, words_ in topics.items():
    top_words[topic] = words_.nlargest(n_words).index.tolist()
pd.DataFrame(top_words)

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

from nltk.corpus import stopwords
stop_words = stopwords.words("russian")

df_state = pd.read_csv('https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/statistics/frequency_state_groups.csv')
df_state = df_state.reset_index()

df_museum = pd.read_csv('https://raw.githubusercontent.com/componavt/sns4human/refs/heads/main/data/vk/statistics/frequency_museum_groups.csv')
df_museum = df_museum.reset_index()